In [1]:
import tensair_aux as tensair
import os
import tensorflow as tf

from tensorflow.keras import Model
from tensorflow.keras import layers
from tensorflow.python.keras.engine import data_adapter
from tensorflow.python.eager import backprop

## Define TensorFlow Model

One may create a TensorFlow model as usual but extending TensAIR instead of Model.

In [2]:
class Cifar(tensair.TensAIR):
    def __init__(self):
        super(Cifar, self).__init__()
        self.l1 = layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3))
        self.l2 = layers.MaxPooling2D((2, 2))
        self.l3 = layers.Conv2D(64, (3, 3), activation='relu')
        self.l4 = layers.MaxPooling2D((2, 2))
        self.l5 = layers.Conv2D(64, (3, 3), activation='relu')
        self.l6 = layers.Flatten()
        self.l7 = layers.Dense(64, activation='relu')
        self.l8 = layers.Dense(10)
        
    @tf.function
    def call(self, x):
        x = self.l1(x)
        x = self.l2(x)
        x = self.l3(x)
        x = self.l4(x)
        x = self.l5(x)
        x = self.l6(x)
        x = self.l7(x)
        x = self.l8(x)
        return x

In [3]:
#compile TensorFlow model as usual
model = Cifar()

learning_rate=0.01
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
opt = tf.keras.optimizers.Adam()

model.compile(optimizer=opt,
              loss=loss,
              metrics=['accuracy'])

# Prepare model for deployment

To prepare the model for deployment, we initialize the TF graph making a test prediction.

At last, we initialize the gradients that will be broadcasted via TensAIR (mandatory)

In [4]:
x = tf.zeros((1, 32, 32, 3), dtype=float)
model.predict(x)

1/1 [==============================] - 0s 123ms/step


2024-03-06 18:46:30.900965: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32)

In [5]:
model.init_delta(model.trainable_weights)

# Save Model

To save the model, we simply define the dimentions and types of the input tensors used during training, a pass those (along with the model) to the tensair.define_signatures function) to obtain the signature of the functions

At last, we save the model on the desired location uwing the signatures previously obtained

In [6]:
input_tensors_dims = [[None,32,32,3],[None]]
input_tensors_types = [tf.float32,tf.int32]
input_tensors_structure = (1,1)

In [7]:
signatures = tensair.define_signatures(model, input_tensors_dims,input_tensors_types,input_tensors_structure)

In [8]:
tensair_path = os.environ.get("TENSAIR_PATH")

In [9]:
model.save(tensair_path+"/data/cifar/cifar_model.tf", save_format="tf", signatures=signatures)

INFO:tensorflow:Assets written to: /Users/mauro.dalleluccatosi/Documents/GitHub-personal/TensAIR/data/cifar/cifar_model.tf/assets


INFO:tensorflow:Assets written to: /Users/mauro.dalleluccatosi/Documents/GitHub-personal/TensAIR/data/cifar/cifar_model.tf/assets
